In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\user\appdata\local\temp\pip-req-build-h2dba6y8


  ERROR: Error [WinError 2] 지정된 파일을 찾을 수 없습니다 while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [2]:
#### kobert 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

ERROR: Invalid requirement: "'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer"
Hint: = is not a valid operator. Did you mean == ?
'subdirectory'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import tqdm
import time
import numpy as np

In [4]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")


ModuleNotFoundError: No module named 'kobert_tokenizer'

In [ ]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1',output_hidden_states = True)
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

In [ ]:
# from kobert import get_tokenizer
# from kobert import get_pytorch_kobert_model

# model, vocab  = get_pytorch_kobert_model()

In [ ]:
# from kobert_tokenizer import KoBERTTokenizer
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [ ]:
# from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')

In [ ]:
# from kobert_tokenizer import KoBERTTokenizer

# MODEL_NAME = 'skt/kobert-base-v1'
# tokenizer = KoBERTTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# ###KoBERT모델 불러오기
# from transformers import BertModel

# model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=True)

In [ ]:
model.to('cuda')

In [ ]:
max_len = 512

In [ ]:
def tokenizer_(text, max_len, tokenizer):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    input_id = encoded_dict['input_ids']
    token_type_id = encoded_dict['token_type_ids']
    attention_mask = encoded_dict['attention_mask']
    
    return input_id, token_type_id, attention_mask

In [ ]:
def bert_embedding(text):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    tokens_tensor = torch.tensor([encoded_dict['input_ids']]).to('cuda')  
    segment_tensors = torch.tensor([encoded_dict['token_type_ids']]).to('cuda')
    attention_tensors = torch.tensor([encoded_dict['attention_mask']]).to('cuda')
    
    model.eval()

    with torch.no_grad():
        outputs = model(tokens_tensor, attention_tensors, segment_tensors)
        
    hidden_states = outputs[2]
    
    token_mean = []

    for h in hidden_states[-4:]:
        token_mean.append(torch.mean(h[0], dim=0))

    last_four_sentence_embedding = sum(token_mean)
    
    return last_four_sentence_embedding.cpu().numpy()

In [ ]:
df = pd.read_excel(r"/content/drive/MyDrive/PythonWorkSpace/이아영_학술대회/추가실험/관세용어사전_길이순.xlsx", index_col = 0)

In [ ]:
df

In [ ]:
kobert_embed_word = df['단어이름'].apply(lambda x: bert_embedding(x))

#KoBERT (monologg/kobert)

In [ ]:
!pip install transformers

In [ ]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 


# Define the model repo
model_name = "monologg/kobert" 


# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Transform input tokens 
inputs = tokenizer("Hello world!", return_tensors="pt")

# Model apply
outputs = model(**inputs)

In [ ]:
model.to('cuda')

In [ ]:
max_len = 512

In [ ]:
def tokenizer_(text, max_len, tokenizer):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    input_id = encoded_dict['input_ids']
    token_type_id = encoded_dict['token_type_ids']
    attention_mask = encoded_dict['attention_mask']
    
    return input_id, token_type_id, attention_mask

In [ ]:
def bert_embedding(text):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    tokens_tensor = torch.tensor([encoded_dict['input_ids']]).to('cuda')  
    segment_tensors = torch.tensor([encoded_dict['token_type_ids']]).to('cuda')
    attention_tensors = torch.tensor([encoded_dict['attention_mask']]).to('cuda')
    
    model.eval()

    with torch.no_grad():
        outputs = model(tokens_tensor, attention_tensors, segment_tensors)
        
    hidden_states = outputs[2]
    
    token_mean = []

    for h in hidden_states[-4:]:
        token_mean.append(torch.mean(h[0], dim=0))

    last_four_sentence_embedding = sum(token_mean)
    
    return last_four_sentence_embedding.cpu().numpy()

In [ ]:
text = df["단어 설명"][0]

In [ ]:
tokenizer(
    text,
    return_token_type_ids=False,
    return_attention_mask=False
)

In [ ]:
print(tokenizer.convert_ids_to_tokens([2, 0, 5330, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6016, 0, 0, 0, 0, 7088, 0, 6116, 0, 7043, 5760, 0, 0, 0, 5944, 0, 7043, 0, 0, 0, 7096, 0, 7043, 0, 0, 0, 0, 7043, 0, 0, 0, 0, 0, 7125, 0, 0, 0, 5548, 6555, 0, 0, 7142, 0, 0, 0, 0, 0, 6555, 0, 0, 0, 0, 7043, 5330, 0, 0, 7125, 0, 0, 0, 0, 7142, 7043, 5859, 6555, 0, 0, 0, 7142, 0, 6502, 7096, 7043, 5760, 0, 0, 0, 0, 0, 6629, 0, 6484, 0, 0, 0, 0, 0, 6555, 0, 0, 0, 0, 0, 5655, 7043, 5760, 6555, 0, 6533, 0, 0, 0, 7096, 0, 5330, 0, 0, 0, 0, 5850, 0, 7127, 0, 0, 0, 0, 6706, 0, 0, 0, 0, 0, 0, 0, 0, 5760, 0, 0, 0, 0, 0, 0, 0, 6555, 0, 0, 0, 0, 0, 7788, 0, 3]))

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/아영/제주학회/관세용어사전_길이순.xlsx', index_col = 0)

In [ ]:
df

In [ ]:
import torch

In [ ]:
kobert_embed = df['단어이름'].apply(lambda x: bert_embedding(x))